In [288]:
import numpy as np
from docplex.mp.model import Model

# from docplex.mp.callbacks.cb_mixin import ConstraintCallbackMixin
# from docplex.mp.callbacks.cb_mixin import LazyConstraintCallback
import matplotlib.pyplot as plt

In [289]:
import cplex
from cplex.callbacks import UserCutCallback, LazyConstraintCallback

In [290]:
# 需要点と施設候補の座標データ
demand_points = [
    (2, 22),
    (42, 6),
    (48, 50),
    (32, 40),
    (16, 10),
    (10, 34),
    (8, 30),
    (29, 16),
    (44, 34),
    (3, 16),
]
candidate_sites = [
    (36, 32),
    (38, 32),
    (44, 11),
    (26, 50),
    (22, 28),
    (32, 46),
    (20, 37),
    (35, 26),
    (2, 4),
    (18, 1),
]

In [291]:
J = len(candidate_sites)  # 施設候補の数
D = len(demand_points)  # 需要点の数
alpha = 0
beta = 0.1

p = 2
r = 2

In [292]:
h_i = np.full(D, 1 / D)

In [293]:
# 既存のリーダーの施設セット J_L を仮定
J_L = {0, 2}  # インデックスとして候補施設の一部を選択
J_F = {1}

In [294]:
# ユークリッド距離の計算
def compute_distances(demand_points, candidate_sites):
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d][j] = np.sqrt(
                (demand_points[d][0] - candidate_sites[j][0]) ** 2
                + (demand_points[d][1] - candidate_sites[j][1]) ** 2
            )
    return distances


distances = compute_distances(demand_points, candidate_sites)

In [295]:
# w_ij の計算
wij_matrix = np.exp(alpha - beta * distances)

In [296]:
def compute_Ui_L(wij_matrix, J_L):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    return wij_matrix[:, list(J_L)].sum(axis=1)


def compute_Ui_F(wij_matrix, J_F):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    return wij_matrix[:, list(J_F)].sum(axis=1)


# U_i^L の計算
Ui_L = compute_Ui_L(wij_matrix, J_L)

# U_i^F の計算
Ui_F = compute_Ui_F(wij_matrix, J_F)

print(f"Ui_L: {Ui_L}")
print(f"Ui_F: {Ui_F}")

Ui_L: [0.04191217 0.65298119 0.13477363 0.45219075 0.1118403  0.09019735
 0.07744449 0.38013697 0.5386599  0.04161902]
Ui_F: [0.02384196 0.072036   0.12756541 0.36787944 0.04454436 0.0603778
 0.04945662 0.15949288 0.53128561 0.02131432]


In [297]:
def compute_L(h_i, Ui_L, Ui_F, wij, x, y):
    """
    関数 L(x, y) を計算する

    Parameters:
        h (np.array): 需要点ごとの人口密度ベクトル (D,)
        Ui_L (np.array): 各需要点におけるリーダーの影響度 (D,)
        Ui_F (np.array): 各需要点におけるフォロワーの影響度 (D,)
        wij (np.array): 需要点と施設候補の重み行列 (D, J)
        x (np.array): リーダーが選択した施設配置 (J,)
        y (np.array): フォロワーが選択した施設配置 (J,)

    Returns:
        float: L(x, y) の計算結果
    """
    numerator = Ui_L + (wij @ x)  # 分子: リーダーの影響度 + 選択した施設の影響
    denominator = Ui_L + Ui_F + (wij @ np.maximum(x, y))  # 分母: 総合影響度

    return np.sum(h_i * (numerator / denominator))


x = np.random.randint(0, 2, J)  # ランダムなリーダーの施設選択 (0 or 1)
y = np.random.randint(0, 2, J)  # ランダムなフォロワーの施設選択 (0 or 1)

print(f"x: {x}")
print(f"y: {y}")

# L(x, y) の計算
L_value = compute_L(h_i, Ui_L, Ui_F, wij_matrix, x, y)

# 計算結果の表示
print(f"L(x, y): {L_value}")

x: [0 0 0 0 1 1 1 0 1 0]
y: [0 0 0 1 1 1 1 0 0 1]
L(x, y): 0.7155875063040147


### 大枠から作っていきますか

In [298]:
import cplex


def solve_s_cflp():
    """S-CFLPのBranch-and-Cutフレームワークを実装"""
    # 初期化
    initialize_problem()

    # Branch-and-Cutのループ
    while True:
        # 連続緩和問題を解く
        incumbent_solution = solve_relaxation()

        # 分離問題を解く
        y_star = separation_problem(incumbent_solution)

        # カットを追加するかチェック
        if should_add_cut(incumbent_solution, y_star):
            add_cuts(incumbent_solution, y_star)
        else:
            # 最適整数解なら終了
            if is_integer_solution(incumbent_solution):
                update_best_solution(incumbent_solution)
                break
            else:
                # 分枝処理を行う
                branch_and_bound(incumbent_solution)

In [299]:
def initialize_problem():
    """
    Branch-and-Cutフレームワークの初期化を行う。
    連続緩和問題をセットアップし、Fに追加する。
    """
    problem = cplex.Cplex()

    # **最大化問題に設定**
    problem.objective.set_sense(problem.objective.sense.maximize)  # ここを明示的に指定

    # 変数 x_j の追加（リーダーの施設配置、0 ≤ x_j ≤ 1）
    x_names = [f"x{j}" for j in range(J)]
    x_types = [problem.variables.type.continuous] * J  # 連続変数
    x_lb = [0] * J  # 下限
    x_ub = [1] * J  # 上限
    x_obj = [0] * J  # 初期の目的関数係数（後で更新）

    # 目的変数 θ の追加
    theta_name = "theta"
    problem.variables.add(
        names=[theta_name],
        types=[problem.variables.type.continuous],  # 連続変数
        lb=[0],  # 下限 0
        ub=[1],  # 上限 1
        obj=[1],  # **目的関数: max θ**
    )

    # 連続変数 x の追加
    problem.variables.add(names=x_names, types=x_types, lb=x_lb, ub=x_ub, obj=x_obj)

    # 施設数制約: Σ x_j = p
    problem.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=x_names, val=[1.0] * J)],
        senses=["E"],  # "E" は equal (=)
        rhs=[p],  # 右辺
        names=["facility_limit"],
    )

    # 初期解のセットアップ
    F = [problem]  # 定式化のセット
    BestSol = None  # 最良解の初期化
    θ_LB = 0  # 最良値の初期化

    return F, BestSol, θ_LB

In [300]:
def solve_relaxation(F):
    """
    連続緩和問題を解き、インカンベント解 (x_hat, theta_hat) を取得する。

    Parameters:
        F (list): 定式化のセット（Cplex 問題オブジェクトのリスト）

    Returns:
        dict: インカンベント解 {'x': np.array, 'theta': float}
    """
    # F から最新の Cplex 問題オブジェクトを取得
    problem = F[-1]

    # 問題を解く
    problem.solve()

    # CPLEX の解の取得（変数名のアンダースコアを削除）
    x_values = np.array(
        problem.solution.get_values([f"x{j}" for j in range(J)])
    )  # "x_0" → "x0"
    theta_value = problem.solution.get_values("theta")

    # インカンベント解を辞書形式で返す
    return {"x": x_values, "theta": theta_value}

一旦上記2つのコードが正しいか、ちゃんと動くかチェックする

In [301]:
# 初期化
F, BestSol, θ_LB = initialize_problem()

# 初期化チェック
print("=== Initialization Check ===")
print(f"Number of Formulations in F: {len(F)}")
print(f"BestSol: {BestSol}")
print(f"θ_LB: {θ_LB}")

# Cplexオブジェクトが作成されているかチェック
# Cplexオブジェクトが作成されているかチェック
problem = F[-1]
print(f"Objective Sense Value: {problem.objective.get_sense()}")
print(f"Maximize Constant: {problem.objective.sense.maximize}")
print(f"Minimize Constant: {problem.objective.sense.minimize}")

# 修正: `get_sense()` を使い、-1 (Maximize) か 1 (Minimize) かチェック
# Maximize の場合 -1、Minimize の場合 1 なので、明示的に -1 と比較する
print(
    f"Objective Sense: {'Maximize' if problem.objective.get_sense() == -1 else 'Minimize'}"
)


# 連続緩和問題を解く
solution = solve_relaxation(F)

# 解のチェック
print("\n=== Relaxation Solution Check ===")
print(f"x values: {solution['x']}")
print(f"θ value: {solution['theta']}")

# x の値が [0,1] の範囲内かチェック
assert np.all(solution["x"] >= 0) and np.all(
    solution["x"] <= 1
), "x values are out of range!"

# θ の値が [0,1] の範囲内かチェック
assert 0 <= solution["theta"] <= 1, "theta value is out of range!"

print("\nAll tests passed successfully!")

=== Initialization Check ===
Number of Formulations in F: 1
BestSol: None
θ_LB: 0
Objective Sense Value: -1
Maximize Constant: -1
Minimize Constant: 1
Objective Sense: Maximize
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 11 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.00 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.01 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.01 ticks)

=== Relaxation Solution Check ===
x values: [1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
θ value: 1.0

All tests passed successfully!


In [302]:
import numpy as np


def separation_problem(incumbent_solution):
    """
    分離問題を解く。Proposition 5 に基づき、最適なフォロワーの施設選択 ŷ を決定する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}

    Returns:
        np.array: 最適なフォロワーの施設配置 y_hat (J,)
    """
    # インカンベント解 (x_hat)
    x_hat = incumbent_solution["x"]

    # 定数 a_i(x_hat) の計算
    a_i = Ui_L + (wij_matrix @ x_hat)

    # 定数 w_i^L(x_hat) および w_i^U(x_hat) の計算
    w_i_L = np.min(Ui_F[:, np.newaxis] + wij_matrix * (1 - x_hat), axis=1)
    w_i_U = np.max(Ui_F[:, np.newaxis] + wij_matrix * (1 - x_hat), axis=1)

    # β(x_hat) の計算
    beta_hat = np.sum(
        h_i[:, np.newaxis]
        * (
            (a_i[:, np.newaxis] * wij_matrix * (1 - x_hat))
            / (
                (a_i[:, np.newaxis] + w_i_U[:, np.newaxis])
                * (a_i[:, np.newaxis] + w_i_L[:, np.newaxis])
            )
        ),
        axis=0,
    )

    # β(x_hat) の降順ソート
    sorted_indices = np.argsort(-beta_hat)  # 降順ソート

    # フォロワーの最適な選択 y_hat
    y_hat = np.zeros(J)
    y_hat[sorted_indices[:r]] = 1  # 上位 r 個を選択

    return y_hat

これも正しく動いているかチェック

In [303]:
y_hat = separation_problem(solution)
print(f"y_hat: {y_hat}")

y_hat: [0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]


In [304]:
def should_add_cut(incumbent_solution, y_star):
    """
    カットを追加すべきか判定

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)

    Returns:
        bool: カットを追加すべきなら True, そうでなければ False
    """
    # インカンベント解 (x_hat, theta_hat)
    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    # L(x_hat, y_star) を計算
    L_value = compute_L(h_i, Ui_L, Ui_F, wij_matrix, x_hat, y_star)

    # カット追加判定
    return theta_hat > L_value

この関数もCheck

In [305]:
bool = should_add_cut(solution, y_hat)
print(f"Should add cut: {bool}")

Should add cut: True


In [ ]:
import itertools
import cplex
import numpy as np

import itertools


def generate_subsets(J):
    """
    J のすべての部分集合 S ⊆ J を生成する（全探索）

    Parameters:
        J (int): 施設候補の数（candidate_sites の長さ）

    Returns:
        list: すべての部分集合 S のリスト（各要素は set 型）
    """
    candidate_indices = set(range(J))  # 修正: `J` を `int` として受け取る
    subsets = []

    # J のすべての部分集合を生成（空集合から J まで）
    for r in range(len(candidate_indices) + 1):
        for subset in itertools.combinations(candidate_indices, r):
            subsets.append(set(subset))  # `set` 型で格納

    return subsets


def compute_L_Y(S, Y):
    """
    L_Y(S) を計算する

    Parameters:
        S (set): リーダーが開設する施設の部分集合
        Y (set): フォロワーが開設する施設の集合

    Returns:
        float: L_Y(S)
    """
    f_i_Y = (Ui_L[:, np.newaxis] + wij_matrix[:, list(S)].sum(axis=1)) / (
        Ui_L[:, np.newaxis]
        + Ui_F[:, np.newaxis]
        + wij_matrix[:, list(S.union(Y))].sum(axis=1)
    )
    return np.sum(h_i * f_i_Y)


def compute_rho_Y(S, k, Y):
    """
    rho_Y(S, k) を計算する

    Parameters:
        S (set): リーダーが開設する施設の部分集合
        k (int): 追加する施設のインデックス
        Y (set): フォロワーの施設配置

    Returns:
        float: rho_Y(S, k)
    """
    return compute_L_Y(S.union({k}), Y) - compute_L_Y(S, Y)

In [ ]:
def add_cuts(incumbent_solution, y_star):
    """
    カット（制約）を追加する。Algorithm 1 に基づき、サブモジュラー不等式 (8) を適用し、
    強化した定式化を F に戻す。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)
    """
    # インカンベント解 (x_hat, theta_hat)
    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    # フォロワーが選んだ施設の集合 Y
    Y = {j for j in range(J) if y_star[j] == 1}

    # 候補施設の集合をセット型で定義
    J_set = set(range(J))

    # 最も違反する S を探索
    min_cut_value = float("inf")
    best_S = None

    for S in generate_subsets(J):
        S_set = set(S)
        cut_value = (
            compute_L_Y(S_set, Y)
            - sum(compute_rho_Y(J_set - {k}, k, Y) * (1 - x_hat[k]) for k in S_set)
            + sum(compute_rho_Y(S_set, k, Y) * x_hat[k] for k in (J_set - S_set))
        )

        if cut_value < min_cut_value:
            min_cut_value = cut_value
            best_S = S_set

    # CPLEX 問題の取得
    problem = F[-1]

    # サブモジュラー不等式 (8) に基づくカットを追加
    if best_S is not None:
        x_coeffs_neg = {k: -compute_rho_Y(J_set - {k}, k, Y) for k in best_S}
        x_coeffs_pos = {k: compute_rho_Y(best_S, k, Y) for k in (J_set - best_S)}

        submodular_cut_expr = cplex.SparsePair(
            ind=[f"x{k}" for k in J_set],
            val=list(x_coeffs_neg.values()) + list(x_coeffs_pos.values()),
        )

        # 修正: `rhs_value` を出力しない
        constraint_str = "θ ≤ "
        first = True
        for k, coeff in x_coeffs_neg.items():
            if not first:
                constraint_str += " + "
            constraint_str += f"({coeff:.4f})x_{k}"
            first = False
        for k, coeff in x_coeffs_pos.items():
            if not first:
                constraint_str += " + "
            constraint_str += f"({coeff:.4f})x_{k}"
            first = False

        print("\n=== Added Submodular Cut ===")
        print(constraint_str)  # 修正: `≤ rhs_value` を削除
        print("==============================\n")

        # CPLEX に制約を追加
        problem.linear_constraints.add(
            lin_expr=[submodular_cut_expr],
            senses=["L"],
            rhs=[min_cut_value],
            names=[f"cut_submodular_{len(problem.linear_constraints.get_names())}"],
        )

    # 強化した定式化を F に戻す
    # F.append(problem)

どんな制約が返ってくるかチェック

In [308]:
add_cuts(solution, y_hat)


=== Added Submodular Cut ===
θ ≤ (-0.0566)x_0 + (0.6659)x_1 + (0.5503)x_2 + (0.4366)x_3 + (2.1268)x_4 + (0.5426)x_5 + (1.7411)x_6 + (0.7419)x_7 + (0.7222)x_8 + (0.7118)x_9



In [309]:
def is_integer_solution(incumbent_solution):
    """インカンベント解が整数解かどうか判定"""
    pass


def update_best_solution(incumbent_solution):
    """最良解を更新する"""
    pass


def branch_and_bound(incumbent_solution):
    """分枝限定法で探索を進める"""
    pass